In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_random_seed(7)

Seed: 7


In [3]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [4]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [5]:
mlp = MLPClassifier(random_state=7)
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000]
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [6]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [7]:
hatebert_dynahate_train_embeddings = reshape_embeddings(hatebert_dynahate_train_embeddings)
hatebert_dynahate_dev_embeddings = reshape_embeddings(hatebert_dynahate_dev_embeddings)
hatebert_dynahate_test_embeddings = reshape_embeddings(hatebert_dynahate_test_embeddings)

hatebert_latenthatred_train_embeddings = reshape_embeddings(hatebert_latenthatred_train_embeddings)
hatebert_latenthatred_dev_embeddings = reshape_embeddings(hatebert_latenthatred_dev_embeddings)
hatebert_latenthatred_test_embeddings = reshape_embeddings(hatebert_latenthatred_test_embeddings)

hatebert_olid_train_embeddings = reshape_embeddings(hatebert_olid_train_embeddings)
hatebert_olid_dev_embeddings = reshape_embeddings(hatebert_olid_dev_embeddings)
hatebert_olid_test_embeddings = reshape_embeddings(hatebert_olid_test_embeddings)

### HateBERT

#### DynaHate

In [8]:
hatebert_dynahate_train_dev = np.concatenate((hatebert_dynahate_train_embeddings, hatebert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(hatebert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_dynahate_train_dev)
test_preds = mlp.predict(hatebert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/hatebert_dynahate_seed_7")
save_model(mlp, "Saves/hatebert_dynahate_seed_7")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7040568280034573
Accuracy Test:  0.6859223300970874
Weighted F1 Train Dev:  0.7016156398708624
Weighted F1 Test:  0.6834324407120073
Macro F1 Train Dev:  0.6985435086620486
Macro F1 Test:  0.6785000884063246
Micro F1 Train Dev:  0.7040568280034573
Micro F1 Test:  0.6859223300970874
Weighted Recall Train Dev:  0.7040568280034573
Weighted Recall Test:  0.6859223300970874
Macro Recall Train Dev:  0.6978285608672722
Macro Recall Test:  0.6774877057454451
Micro Recall Train Dev:  0.7040568280034573
Micro Recall Test:  0.6859223300970874
Confusion Matrix Train Dev: 
[[10530  6587]
 [ 4370 15537]]
Confusion Matrix Test: 
[[1100  752]
 [ 542 1726]]


#### LatentHatred

In [9]:
hatebert_latenthatred_train_dev = np.concatenate((hatebert_latenthatred_train_embeddings, hatebert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(hatebert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_latenthatred_train_dev)
test_preds = mlp.predict(hatebert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/hatebert_latenthatred_seed_7")
save_model(mlp, "Saves/hatebert_latenthatred_seed_7")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6908131595282433
Accuracy Test:  0.6761638733705773
Weighted F1 Train Dev:  0.6583417391428766
Weighted F1 Test:  0.6402297198205446
Macro F1 Train Dev:  0.44868676889483816
Macro F1 Test:  0.4202525953335389
Micro F1 Train Dev:  0.6908131595282433
Micro F1 Test:  0.6761638733705773
Weighted Recall Train Dev:  0.6908131595282433
Weighted Recall Test:  0.6761638733705773
Macro Recall Train Dev:  0.4469605855343905
Macro Recall Test:  0.425545795533222
Micro Recall Train Dev:  0.6908131595282433
Micro Recall Test:  0.6761638733705773
Confusion Matrix Train Dev: 
[[8839 1094   12]
 [3070 2267   10]
 [ 416  379   23]]
Confusion Matrix Test: 
[[2949  396    1]
 [1071  680    2]
 [ 154  115    2]]


#### OLID

In [10]:
hatebert_olid_train_dev = np.concatenate((hatebert_olid_train_embeddings, hatebert_olid_dev_embeddings))

grid_results = gridsearch.fit(hatebert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_olid_train_dev)
test_preds = mlp.predict(hatebert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/hatebert_olid_seed_7")

save_model(mlp, "Saves/hatebert_olid_seed_7")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7462235649546828
Accuracy Test:  0.7895348837209303
Weighted F1 Train Dev:  0.7293536795911543
Weighted F1 Test:  0.7627746964608499
Macro F1 Train Dev:  0.6809671866880105
Macro F1 Test:  0.6801009472020649
Micro F1 Train Dev:  0.7462235649546827
Micro F1 Test:  0.7895348837209303
Weighted Recall Train Dev:  0.7462235649546828
Weighted Recall Test:  0.7895348837209303
Macro Recall Train Dev:  0.6698920197449609
Macro Recall Test:  0.6599462365591398
Micro Recall Train Dev:  0.7462235649546828
Micro Recall Test:  0.7895348837209303
Confusion Matrix Train Dev: 
[[7934  906]
 [2454 1946]]
Confusion Matrix Test: 
[[591  29]
 [152  88]]
